In [18]:
import sys
import json
default = {'training_folder': '../../'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../'}


In [19]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
from typing import Literal

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [20]:
base_folder = argv['training_folder']

In [21]:
name = 'Houston_CT'
zones = gpd.read_file(os.path.join(base_folder,'inputs/zones', name+'.geojson'), engine=io_engine)  


In [22]:
zones = zones.set_index('OBJECTID')
zones.index.name = 'index'

In [23]:
zones = zones.to_crs(4326)

In [24]:
zones['County']

index
1       Austin County
2       Austin County
3       Austin County
4       Austin County
5       Austin County
            ...      
1607    Waller County
1608    Waller County
1609    Waller County
1610    Waller County
1611    Waller County
Name: County, Length: 1611, dtype: object

In [25]:
zones[['P001_calc_pctPopDensity','P012_calc_pctWork','P012_calc_numWork','County']]

,P001_calc_pctPopDensity,P012_calc_pctWork,P012_calc_numWork,County
index,,,,
1,14.0,58.5,1442,Austin County
2,28.6,60.8,1264,Austin County
3,56.9,58.6,2606,Austin County
4,26.5,54.4,1509,Austin County
5,19.8,57.7,2910,Austin County
...,...,...,...,...
1607,32.3,59.0,2880,Waller County
1608,18.2,57.0,2474,Waller County
1609,39.5,56.4,1488,Waller County


In [26]:
tmp_zones = zones[['P001_calc_pctPopDensity','P0010001','P012_calc_numWork','geometry','County']].rename(columns={'P001_calc_pctPopDensity':'density', 'P0010001':'population', 'P012_calc_numWork': 'numWork'  })

In [27]:
output_folder = base_folder+'scenarios/houston/inputs/'
tmp_zones.to_file(output_folder + 'zones.geojson',driver='GeoJSON')

In [28]:
for col in zones.columns:
    if 'wo' in col.lower():
        print(col)

P012_calc_pctWork
P012_calc_numWork


In [29]:
zones[['P001_calc_pctPopDensity','P012_calc_pctWork','P012_calc_numWork']]

,P001_calc_pctPopDensity,P012_calc_pctWork,P012_calc_numWork
index,,,
1,14.0,58.5,1442
2,28.6,60.8,1264
3,56.9,58.6,2606
4,26.5,54.4,1509
5,19.8,57.7,2910
...,...,...,...
1607,32.3,59.0,2880
1608,18.2,57.0,2474
1609,39.5,56.4,1488


In [30]:
zones['Shape_Area']

index
1       2.347989e+08
2       9.842046e+07
3       1.054651e+08
4       1.415368e+08
5       3.445981e+08
            ...     
1607    2.058521e+08
1608    3.248553e+08
1609    8.959761e+07
1610    1.875881e+08
1611    6.663682e+06
Name: Shape_Area, Length: 1611, dtype: float64

In [31]:
# find meters CRS
from shapely.geometry import Point, LineString
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

centroid = [*LineString(zones.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

/tmp/ipykernel_19182/2198738084.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(zones.centroid.values).centroid.coords][0]


32615

In [32]:
zones['area (km2)'] = zones.to_crs(crs).area
zones['area (km2)'].sum()

23286871095.244537

In [33]:
zones[['Shape_Area','area (km2)']]

,Shape_Area,area (km2)
index,,
1,2.347989e+08,1.769315e+08
2,9.842046e+07,7.393711e+07
3,1.054651e+08,7.933958e+07
4,1.415368e+08,1.062743e+08
5,3.445981e+08,2.591938e+08
...,...,...
1607,2.058521e+08,1.541194e+08
1608,3.248553e+08,2.423717e+08
1609,8.959761e+07,6.686994e+07


In [34]:
zones['Shape_Area']/zones['area (km2)']

index
1       1.327061
2       1.331137
3       1.329288
4       1.331807
5       1.329500
          ...   
1607    1.335666
1608    1.340318
1609    1.339879
1610    1.341663
1611    1.338709
Length: 1611, dtype: float64